<!-- Trabalho Desenvolvido no Curso da Data Science Academy - www.datascienceacademy.com.br -->
# Data Science Academy
# Fundamentos de Linguagem Python - Do Básico a Aplicações de IA
# Mini-Projeto 4
## Limpeza, Engenharia de Atributos e Análise Exploratória de Dados de Vendas com Pandas

## 1. Definição do Problema de Negócio

### 1.1. Contexto 

Uma empresa de e-commerce, em plena fase de crescimento, coleta diariamente um volume significativo de dados transacionais de suas vendas online. Esses dados, provenientes de diversas fontes, são armazenados em um sistema bruto e incluem informações sobre pedidos, clientes, produtos, categorias e status de entrega.

Atualmente, a gestão da empresa enfrenta dificuldades para tomar decisões estratégicas baseadas em dados, pois os relatórios gerados a partir desse sistema bruto são inconsistentes e pouco confiáveis. Tentativas de analisar o desempenho de produtos ou entender o comportamento do consumidor resultaram em conclusões contraditórias, gerando incerteza nas áreas de marketing, estoque e planejamento financeiro.

### 1.2. Problema de Negócio 

A empresa não consegue extrair inteligência de negócio de seus dados de vendas devido à baixa qualidade e à falta de estruturação das informações. Os dados brutos contêm problemas recorrentes, como:

- Dados Faltantes: Pedidos sem registro de quantidade ou status, impedindo um cálculo preciso do faturamento e da eficiência logística.

- Inconsistências e Erros: Formatos de dados incorretos (ex: preços como texto) e erros de digitação que corrompem as análises.

- Informações Duplicadas: Registros de vendas que aparecem mais de uma vez, inflando artificialmente os números de receita e unidades vendidas.

- Valores Discrepantes (Outliers): Dados anômalos que distorcem as médias e as análises estatísticas, levando a uma compreensão equivocada do comportamento padrão de compra.
<!-- Trabalho Desenvolvido no Curso da Data Science Academy - www.datascienceacademy.com.br -->

Essa falta de dados confiáveis impede a empresa de responder a perguntas críticas de negócio, tais como:

- Quais são nossos produtos e categorias mais lucrativos?

- Qual é a nossa tendência de vendas ao longo do tempo?

- Como está a eficiência de nossa operação logística (percentual de entregas, pendências, etc.)?

A incapacidade de responder a essas perguntas resulta em alocação ineficiente de recursos, perda de oportunidades de vendas e dificuldade em planejar o futuro do negócio de forma estratégica.

### 1.3. Objetivos do Projeto 

O objetivo principal deste projeto é transformar os dados brutos de vendas em um ativo confiável e acionável, permitindo que a empresa baseie suas decisões estratégicas em informações precisas.

Para alcançar este objetivo, os seguintes passos serão executados:

- Limpeza e Validação: Implementar um processo para identificar e corrigir as inconsistências nos dados, tratando valores ausentes, removendo duplicatas, corrigindo tipos de dados e lidando com outliers.

- Análise Exploratória de Dados (EDA): Analisar o conjunto de dados limpo para extrair insights iniciais e responder às perguntas de negócio fundamentais.

- Geração de Insights Visuais: Criar um conjunto de visualizações que comuniquem de forma clara e objetiva os principais indicadores de desempenho de vendas.

### 1.4. Critérios de Sucesso 

O projeto será considerado um sucesso quando formos capazes de entregar:

- Um conjunto de dados (dataset) limpo e validado, pronto para ser utilizado em análises futuras.

- Um relatório de análise respondendo com clareza às seguintes questões:

    - O faturamento total e a receita por categoria de produto.

    - A identificação do top 5 produtos mais vendidos em quantidade e em receita.

    - Um gráfico mostrando a tendência de vendas diárias no período analisado.

    - Uma análise da distribuição percentual dos status de entrega (Entregue, Pendente, Cancelado).

O sucesso final será medido pela capacidade da equipe de gestão de utilizar os insights gerados para tomar, com confiança, pelo menos uma decisão de negócio informada (ex: ajustar o estoque do produto mais vendido ou criar uma campanha de marketing para a categoria mais lucrativa).

## 2. Importação das Bibliotecas

In [1]:
# Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%reload_ext watermark

In [3]:
%watermark --iversions

matplotlib: 3.10.8
numpy     : 2.4.2
pandas    : 3.0.0
seaborn   : 0.13.2



## Passo 1 - Geração de Dados Sintéticos com Pandas e NumPy

Criamos um DataFrame com dados fictícios que simulam problemas comuns: valores ausentes, tipos de dados incorretos, duplicatas e outliers.

In [6]:
# --- Geração de Dados Fictícios Coerentes ---
print("\nGerando conjunto de dados fictícios...")

# Define a semente para resultados reproduzíveis
np.random.seed(42)

# Criando um dicionário de dados
data = {
    'ID_Pedido': range(1001, 1101),
    'Data_Compra': pd.to_datetime(pd.date_range(start = '2026-07-01', periods = 100, freq ='D')) - pd.to_timedelta(np.random.randint(0, 30, size = 100), unit = 'd'),
    'Cliente_ID': np.random.randint(100, 150, size = 100),
    'Produto': np.random.choice(['Smartphone', 'Notebook', 'Fone de Ouvido', 'Smartwatch', 'Teclado Mecânico'], size = 100),
    'Categoria': ['Eletrônicos', 'Eletrônicos', 'Acessórios', 'Acessórios', 'Acessórios'] * 20,
    'Quantidade': np.random.randint(1, 5, size = 100),
    'Preco_Unitario': [5999.90, 8500.00, 799.50, 2100.00, 850.00] * 20,
    'Status_Entrega': np.random.choice(['Entregue', 'Pendente', 'Cancelado'], size = 100, p = [0.8, 0.15, 0.05])
}

# Criando o dataframe a partir do dicionário
df_vendas = pd.DataFrame(data)

# --- Introduzindo Problemas nos Dados para o Exercício ---
print("\nIntroduzindo problemas nos dados para a limpeza...\n")

# 1. Valores Ausentes (NaN)
df_vendas.loc[5:10, 'Quantidade'] = np.nan
df_vendas.loc[20:22, 'Status_Entrega'] = np.nan
df_vendas.loc[30, 'Cliente_ID'] = np.nan

# 2. Dados Duplicados
df_vendas = pd.concat([df_vendas, df_vendas.head(3)], ignore_index = True)

# 3. Tipos de Dados Incorretos
df_vendas['Preco_Unitario'] = df_vendas['Preco_Unitario'].astype(str)
df_vendas.loc[15, 'Preco_Unitario'] = 'valor_invalido'                   # Simulando um erro de digitação
df_vendas['Cliente_ID'] = df_vendas['Cliente_ID'].astype(str)

# 4. Outliers
df_vendas.loc[50, 'Quantidade'] = 50 # Um valor claramente fora do padrão

print("Dados gerados com sucesso!\n")


Gerando conjunto de dados fictícios...

Introduzindo problemas nos dados para a limpeza...

Dados gerados com sucesso!



/tmp/ipykernel_184132/1411456404.py:10: Pandas4Warning: 'd' is deprecated and will be removed in a future version. Please use 'D' instead of 'd'.
  'Data_Compra': pd.to_datetime(pd.date_range(start = '2026-07-01', periods = 100, freq ='D')) - pd.to_timedelta(np.random.randint(0, 30, size = 100), unit = 'd'),


In [7]:
# Primeiras linhas
df_vendas.head()

,ID_Pedido,Data_Compra,Cliente_ID,Produto,Categoria,Quantidade,Preco_Unitario,Status_Entrega
0,1001,2026-06-25,109.0,Smartwatch,Eletrônicos,1.0,5999.9,Cancelado
1,1002,2026-06-13,135.0,Smartwatch,Eletrônicos,3.0,8500.0,Entregue
2,1003,2026-06-05,113.0,Smartwatch,Acessórios,3.0,799.5,Entregue
3,1004,2026-06-20,130.0,Smartwatch,Acessórios,4.0,2100.0,Entregue
4,1005,2026-06-25,147.0,Smartwatch,Acessórios,4.0,850.0,Cancelado


In [8]:
# Últimas linhas
df_vendas.tail()

,ID_Pedido,Data_Compra,Cliente_ID,Produto,Categoria,Quantidade,Preco_Unitario,Status_Entrega
98,1099,2026-09-26,144.0,Fone de Ouvido,Acessórios,4.0,2100.0,Entregue
99,1100,2026-10-07,105.0,Smartphone,Acessórios,1.0,850.0,Entregue
100,1001,2026-06-25,109.0,Smartwatch,Eletrônicos,1.0,5999.9,Cancelado
101,1002,2026-06-13,135.0,Smartwatch,Eletrônicos,3.0,8500.0,Entregue
102,1003,2026-06-05,113.0,Smartwatch,Acessórios,3.0,799.5,Entregue


**Gerar dados com problemas é muito mais fácil do que corrigir os problemas**. Por isso há 2 grupos de pessoas no mundo:

- As que criam problemas!
- As que resolvem problemas!

Fique sempre no segundo grupo, pois a concorrência é muito menor!